# Naive bayesian decoder
Generative classifiers

Conditional probabilities

Tuning curves to probabilities

Bayes rule

Naive bayes classifier with numpy

Naive bayes classifier with sklearn

Noise correlation and normalization



## Generative vs. discriminative classifiers

So far we have used logistic regression classifiers to do our decoding. These take a set of features and use them to predict the probability of the event class that generated it. 

## Conditional probability

We already mentioned conditional probabilities when discussing how to interpret the logistic function. In that case, the logisitic function reflects the probability of a certain event occurring given the features in our data, $p(y|x)$, where $y$ is the event and $x$ are the features. Across all possible classes there will be a different $p(y_k|x)$, and the sum of those probabilities across all possible classes will be 1:

$$ \sum_{k=1}^{K}{p(y_k|x)} = 1 $$

When evaluating the probability across all classes, the $x$ is kept constant. 

The general way to express conditional probability is:

$$ p(y|x) = \frac{p(y,x)}{p(x)} \tag{eq. 1} $$

Here $p(y,x)$ is the probability that particular instances of $y$ and $x$ ever co-occur. If we divide that by the probability of $x$ occurring, $p(x)$, then we have the probability of $y$ *conditional* on $x$. That is, it is the probability of $y$ for all instances when $x$ is of a particular value.

When $y$ and $x$ are conditionally independent of one another then $p(y|x)=p(y)p(x)$. This just means that the chance $y$ occurring with $x$ will equal to the probability that either of those happen in general (irrespective of each other). This is what is meant by conditional independence. 



## Bayes' theorem

Given equation 1 and a little algebra, we can establish a fundamental relationship between conditional probabilities known as Bayes' rule or theorem. Note that swapping $x$ and $y$, we get the equation $p(x|y)=\frac{p(x,y)}{p(y)}$. Also note that $p(y,x) = p(x,y)$. With these facts, we can write:

$$ \begin{align}
    p(y|x)p(x) &= p(y,x) = p(x,y) = p(x|y)p(y) \notag \\
    p(y|x)p(x) &= p(x|y)p(y) \notag \\
    p(y|x) &= \frac{p(x|y)p(y)}{p(x)} \tag{2} \\
    \end{align}
$$

This powerful relationship shows how to go from the conditional probability of $x$ given $y$, to that of $y$ given $x$. 

Why is this relaiontship important? Returning to decoders, their goal is to estimate the probability of behavioral or stimulus events, $y_k$, given patterns in brain activity, $x$. In the previous lectures we estimated this probability, $p(y_k|x)$ using logistic regression. The event $k$ with the largest probability was then chosen as the one most likely to have occured. However, Bayes' theorem offers an alternative approach. Instead we can derive that by measuring the probability of observing our brain data given that event $y_k$ occured and use that, along with our knowledge about how often $y_k$ occurs.

However, we do have 